In [4]:
import pandas as pd
import numpy as np
import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
    
    with warnings.catchwarnings():
        
        warnings.simplefilter('ignore')
        fxn()
data = pd.read_csv('HN_posts_year_to_Sep_26_2016.csv')
print('Shape of dataset is:',data.shape)
data.head()

Shape of dataset is: (293119, 7)


,id,title,url,num_points,num_comments,author,created_at
0,12579008,You have two days to comment if you want stem ...,http://www.regulations.gov/document?D=FDA-2015...,1,0,altstar,9/26/2016 3:26
1,12579005,SQLAR the SQLite Archiver,https://www.sqlite.org/sqlar/doc/trunk/README.md,1,0,blacksqr,9/26/2016 3:24
2,12578997,What if we just printed a flatscreen televisio...,https://medium.com/vanmoof/our-secrets-out-f21...,1,0,pavel_lishin,9/26/2016 3:19
3,12578989,algorithmic music,http://cacm.acm.org/magazines/2011/7/109891-al...,1,0,poindontcare,9/26/2016 3:16
4,12578979,How the Data Vault Enables the Next-Gen Data W...,https://www.talend.com/blog/2016/05/12/talend-...,1,0,markgainor1,9/26/2016 3:14


In [5]:
pd.DataFrame(data.isnull().sum()/data.shape[0]*100,columns=['% Missing Values']).round(2)

,% Missing Values
id,0.00
title,0.00
url,4.73
num_points,0.00
num_comments,0.00
author,0.00
created_at,0.00


In [6]:
print('Highest number of posts is {1} made by {0}'.format(data['author'].value_counts().index.tolist()[0],data['author'].value_counts().tolist()[0]))

Highest number of posts is 2731 made by jonbaer


In [7]:
jonbaer=data[data['author']=='jonbaer']
print('jonbaer received {0:,.2f} average points, while average points for all posts is {1:.2f}'.format(jonbaer['num_points'].mean(),data['num_points'].mean()))

jonbaer received 9.58 average points, while average points for all posts is 15.03


In [8]:
ave_votes_byauthor=data.groupby('author').mean()
ave_votes_byauthor['num_points'].sort_values(ascending=False).head(5)

author
intull         2011.0
alankay1       1401.0
mmebane        1248.0
boren_ave11    1213.0
hannahmitt     1172.0
Name: num_points, dtype: float64

In [22]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

def histly(df,target):
    title_text='Histogram of log of average {0} by user'.format(target)
    
    data1 = [go.Histogram(x=np.log1p(df[target]))]
    shapes_list=[{'type': 'line', 'xref': 'x', 'yref': 'paper', 'x0': np.log1p(df[target].mean()), 'y0':0, 'x1': np.log1p(df[target].mean()), 'y1':1, 'line': {  'width' : 5, 'dash': 'dashdot'}}]
    annotations_list=[{'x' :np.log1p(df[target].mean()), 'y' : 50, 'xref': 'x', 'yref':'y', 'text':'Average across all data', 'showarrow':True, 'arrowhead':7, 'ax':100, 'ay':-100}]
    layout = go.Layout(title=title_text, font=dict(size=14), xaxis={'title':'Log of average', 'titlefont':{'size':18}}, yaxis={'title':'Count', 'titlefont':{'size':18}}, autosize=True, shapes=shapes_list, annotations=annotations_list)
    fig = go.Figure(data=data1, layout=layout)
    py.iplot(fig)

In [23]:
histly(ave_votes_byauthor,'num_points')

In [24]:
train=data.loc[:, ['title','num_points']]
train=train.sample(frac=0.05,axis=0).reset_index()

train=train.dropna()
train.shape

(14656, 3)

In [27]:
import string
train['title_nopuncs']=train['title'].apply(lambda x:x.translate(str.maketrans('','',string.punctuation)))

In [28]:
train['title_nopuncs']=train['title_nopuncs'].apply(lambda x: x.lower())

In [31]:
train['tokenz'] = train['title_nopuncs'].apply(lambda x: x.split())
train['tokenz'].head()

0    [google, faces, eu, antitrust, charges, over, ...
1               [affordable, laser, engraver, machine]
2          [a, sharing, economy, where, teachers, win]
3          [helicopters, and, muscle, memory, science]
4                        [energyefficient, algorithms]
Name: tokenz, dtype: object

In [32]:
import itertools
words=list(itertools.chain.from_iterable(train['tokenz']))
unique_words=list(set(words))
print('Number of unique words:',len(set(unique_words)))

Number of unique words: 18982


In [33]:
counts = pd.DataFrame(0,index=np.arange(train.shape[0]), columns=unique_words)

In [34]:
for index, row in train.iterrows():
    for token in row['tokenz']:
        counts.iloc[index][token]+=1

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(list(train['title']))
counts=pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

In [36]:
count_sum=counts.sum()
counts=counts.drop(count_sum[(count_sum>100) | (count_sum<5)].index,axis=1)

In [39]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(counts,train['num_points'],train_size=0.8,random_state=1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



In [42]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
lr.fit(X_train,y_train)
pred=lr.predict(X_test)
rmse=(mean_squared_error(pred,y_test))**0.5
print('RMSE is: {0:.2f}'.format(rmse))

RMSE is: 60.32
